In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import sqlite3
import glob
import pandas as pd
%matplotlib inline
import seaborn as sns
from helpers_funcs.sql_funcs import fetch_mquote
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error

# Grabbing data from the SQLite Database. 
datas_conn = sqlite3.connect('stocks_data.db')
datas_c = datas_conn.cursor()
datas_c.execute("SELECT name FROM sqlite_master WHERE type='table';")
current_list = (datas_c.fetchall())

Using Theano backend.


In [ ]:
import math

In [ ]:
def dates_df_fromdf(symbol_df):
    #Generates a Series of the dates and timestamps which correspond to
    # Midquote values.
    dates= pd.Series(symbol_df['DATE'])
    dates = pd.to_timedelta(dates, unit='D') + pd.Timestamp('1960-1-1')
    for x in range(len(dates)):
        dates[x]=pd.Timestamp(str(dates[x])[:11]+(symbol_df['itime'][x]))
        
    return dates


In [ ]:
df_KME = pd.read_sql_query('select * from "{}"'.format('KME'), datas_conn)


In [ ]:
df_KME.head(n=5)

In [ ]:
def create_training_set(stock_data, look_back=5):
    dataX, dataY = [], []
    for i in range(len(stock_data)-look_back-1):
        temp_val = stock_data[i:(i+look_back)]
        dataX.append(temp_val)
        dataY.append(stock_data[i + look_back])
    return np.array(dataX), np.array(dataY)
 

In [7]:
def graphing_wrapper(len_test,len_train,train,test,data_df,title_test,title_train):
    fig_MLP, ax_MLP = plt.subplots(figsize=(20,10))
    ax_MLP.plot(data_df['mquote'].values[0:(train_size)],color='#225ea8',label='KME Midquote Values')
    ax_MLP.plot(train,alpha=0.6,color='#a1dab4',label='Train Prediction')
    ax_MLP.set_title(title_test,size=20)
    ax_MLP.set_xlabel('Dates')
    ax_MLP.set_ylabel('MidQuote')
    ax_MLP.legend(prop={'size':18})

    ax_MLP.set_xticklabels(['Sep 2000','Nov 2000','Jan 2001','Mar 2001','May 2001','Jul 2001','Sep 2001','Nov 2001'])
    fig_MLP2, ax_MLP2 = plt.subplots(figsize=(20,10))
    ax_MLP2.plot(data_df['mquote'].values[-(test_size):],color='#225ea8',label='KME Midquote Values')
    ax_MLP2.plot(test,color='#41b6c4',alpha=0.6,label='Test Prediction')
    ax_MLP2.set_xticklabels(['Dec 2001','Jan 2002','Feb 2002','Mar 2002','Apr 2002','May 2002','Jun 2002','Jul 2002'])
    ax_MLP2.set_title(title_train,size=20)
    ax_MLP2.set_xlabel('Dates')
    ax_MLP2.set_ylabel('MidQuote')
    ax_MLP2.legend(prop={'size':18})
    
    xrand = np.linspace(0,10000,len(data_df['mquote'].values))

    fig_MLP3, ax_MLP3 = plt.subplots(figsize=(20,10))
    ax_MLP3.plot(xrand,data_df['mquote'].values,color='#225ea8',label='KME Midquote Values')
    ax_MLP3.plot(xrand[0:train_size],np.concatenate((train,test))[0:(train_size)],alpha=0.6,color='#a1dab4',label='Train Prediction')
    #ax_MLP3.plot(data_df['mquote'].values[-(test_size):],color='#225ea8',label='KME Midquote Values')
    ax_MLP3.plot(xrand[-test_size:],np.concatenate((train,test))[-(test_size):],color='#41b6c4',alpha=0.6,label='Test Prediction')
    ax_MLP3.set_title('KME Training and Test Prediction from Multilayer Perceptron',size=20)
    ax_MLP3.set_xlabel('Dates')
    ax_MLP3.set_ylabel('MidQuote')
    ax_MLP3.legend(prop={'size':18})
    ax_MLP3.set_xticklabels(['Sep 2000','Dec 2000','Mar 2001','Jun 2001','Sep 2001','Dec 2001','Mar 2002','Jun 2002'])

    
    fig_MLP4, ax_MLP4 = plt.subplots(figsize=(20,10))
    ax_MLP4.plot(np.diff(data_df['mquote'].values[0:train_size],n=50),color='#225ea8',label='KME Midquote Values')
    ax_MLP4.plot(np.diff(train,n=50),color='#a1dab4',label='Train Prediction')
    ax_MLP4.set_xticklabels(['Sep 2000','Nov 2000','Jan 2001','Mar 2001','May 2001','Jul 2001','Sep 2001','Nov 2001'])
    ax_MLP4.set_title('Percentage Change in Train Prediction ')
    ax_MLP4.set_xlabel('Dates')
    ax_MLP4.legend(prop={'size':18})
    train_mse = mean_squared_error(data_df['mquote'].values[0:len(train)],train)
    print("Mean Squared Error of Training = ", train_mse)
    
    
    fig_MLP5, ax_MLP5 = plt.subplots(figsize=(20,10))
    ax_MLP5.plot(np.diff(data_df['mquote'].values[-len(test_MLP_predict):],n=50),color='#225ea8',label='KME Midquote Values')
    ax_MLP5.plot(np.diff(test,n=50),color='#41b6c4',label='Test Prediction')
    ax_MLP5.set_xticklabels(['Dec 2001','Jan 2002','Feb 2002','Mar 2002','Apr 2002','May 2002','Jun 2002','Jul 2002'])
    ax_MLP5.set_title('Percentage Change in Test Prediction ')
    ax_MLP5.set_xlabel('Dates')
    ax_MLP5.legend(prop={'size':18})
    test_mse = mean_squared_error(data_df['mquote'].values[-len(test):],test)
    print("Mean Squared Error of Test = ", test_mse)


In [8]:
data_df=df_KME
look_back=1000

In [9]:
train_size=math.floor(len(data_df['mquote'].values)*0.67)
test_size = len(data_df['mquote'].values)-train_size
train_values = data_df['mquote'].values[0:train_size]
test_values = data_df['mquote'].values[train_size:train_size+test_size]
train_X,train_Y = create_training_set(train_values,look_back=look_back)
test_X,test_Y = create_training_set(test_values,look_back=look_back)
model_MLP= Sequential()
model_MLP.add(Dense(16, input_dim=look_back, activation='relu'))
model_MLP.add(Dense(8,activation='relu'))
model_MLP.add(Dense(4,activation='relu'))
model_MLP.add(Dense(1))
model_MLP.compile(loss='mean_squared_error', optimizer='adam')
model_MLP.fit(train_X, train_Y, nb_epoch=20, batch_size=5, verbose=2)
train_evaluation  = model_MLP.evaluate(train_X,train_Y,batch_size=5,verbose=0)
test_evaluation = model_MLP.evaluate(test_X,test_Y,batch_size=5,verbose=0)
print("Train Score:",train_evaluation)
print("Test Score:", test_evaluation)
train_MLP_predict = model_MLP.predict(train_X)
test_MLP_predict = model_MLP.predict(test_X)
train_MLP_predict=np.reshape(train_MLP_predict,train_MLP_predict.shape[0])
test_MLP_predict=np.reshape(test_MLP_predict,test_MLP_predict.shape[0])


Epoch 1/20
50s - loss: 0.0719
Epoch 2/20
53s - loss: 0.0240
Epoch 3/20
52s - loss: 0.0201
Epoch 4/20
58s - loss: 0.0185
Epoch 5/20
67s - loss: 0.0159
Epoch 6/20
68s - loss: 0.0151
Epoch 7/20
67s - loss: 0.0143
Epoch 8/20
67s - loss: 0.0133
Epoch 9/20
68s - loss: 0.0126
Epoch 10/20
67s - loss: 0.0121
Epoch 11/20
68s - loss: 0.0119
Epoch 12/20
67s - loss: 0.0114
Epoch 13/20
67s - loss: 0.0113
Epoch 14/20
67s - loss: 0.0110
Epoch 15/20
67s - loss: 0.0106
Epoch 16/20
65s - loss: 0.0103
Epoch 17/20
64s - loss: 0.0100
Epoch 18/20
67s - loss: 0.0101
Epoch 19/20
67s - loss: 0.0098
Epoch 20/20
67s - loss: 0.0097
Train Score: 0.0128722593823
Test Score: 0.00150314938883


In [10]:
def consolidate_learning_lstm(initial_neurons,hidden_layers,train_percent,test_percent,data_df,look_back,title_train,title_test):
    np.random.seed(7)
    scalar = MinMaxScaler(feature_range=(0, 1))
    normalized_data = data_df['mquote'].values.reshape(len(data_df['mquote'].values),1)
    normalized_data = scalar.fit_transform(normalized_data)
    train_size=math.floor(len(normalized_data)*train_percent)
    test_size = len(normalized_data)-train_size
    train_values = normalized_data[0:train_size]
    test_values = normalized_data[train_size:train_size+test_size]
    train_X,train_Y = create_training_set(train_values,look_back=look_back)
    test_X,test_Y = create_training_set(test_values,look_back=look_back)
    train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))
    test_X = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))
    model_LSTM= Sequential()
    model_LSTM.add(LSTM(initial_neurons, input_dim=look_back))
    for x in range(hidden_layers):
        num =1
        if(math.floor(hidden_layers/2)==0):
            num =1
        else:
            num = math.floor(hidden_layers/2)
            
        model_LSTM.add(Dense(initial_neurons/num,activation='relu'))
        initial_neurons = initial_neurons/hidden_layers
    model_LSTM.add(Dense(1,activation='sigmoid'))
    model_LSTM.compile(loss='mean_squared_error', optimizer='adam')
    model_LSTM.fit(train_X, train_Y, nb_epoch=50, batch_size=5, verbose=2)
    train_evaluation  = model_LSTM.evaluate(train_X,train_Y,batch_size=5,verbose=0)
    test_evaluation = model_LSTM.evaluate(test_X,test_Y,batch_size=5,verbose=0)
    train_LSTM_predict = model_LSTM.predict(train_X)
    test_LSTM_predict = model_LSTM.predict(test_X)
    train_LSTM_predict = scalar.inverse_transform(train_LSTM_predict)
    train_Y = scalar.inverse_transform(train_Y.reshape(-1, 1))
    test_LSTM_predict = scalar.inverse_transform(test_LSTM_predict)
    test_Y = scalar.inverse_transform(test_Y.reshape(-1, 1))
    #train_LSTM_predict=np.reshape(train_LSTM_predict,train_LSTM_predict.shape[0])
    #test_LSTM_predict=np.reshape(test_LSTM_predict,test_LSTM_predict.shape[0])
    print((train_LSTM_predict.shape))
    
    train_predict_plot = np.empty_like(normalized_data)
    train_predict_plot[:, :] = np.nan
    print((train_predict_plot[look_back:len(train_LSTM_predict)+look_back, :].shape))
    train_predict_plot[look_back:len(train_LSTM_predict)+look_back, :] = train_LSTM_predict
    test_predict_plot = np.empty_like(normalized_data)
    test_predict_plot[:, :] = np.nan
    test_predict_plot[len(train_LSTM_predict)+(look_back*2)+1:len(normalized_data)-1, :] = test_LSTM_predict
    print("Train Score:",train_evaluation)
    print("Test Score:", test_evaluation)
    #train_LSTM_predict_mse = mean_squared_error(data_df['mquote'].values[0:len(train_LSTM_predict)],train_predict_plot[~np.isnan(train_predict_plot)])
    #test_LSTM_predict_mse = mean_squared_error(data_df['mquote'].values[-(test_LSTM_predict):0],test_predict_plot[~np.isnan(tesPredictPlot)])
    #print("Training Mean Squared Error",train_LSTM_predict_mse)
    #print("Testing Mean Squared Error",test_LSTM_predict_mse)
    #plt.plot(test_predict_plot)
    #plt.plot(train_predict_plot)
    #graphing_wrapper_LSTM(dates_df_fromdf_2(data_df),train_size,test_size,train_predict_plot,test_predict_plot,data_df,title_train,title_test)

In [11]:
consolidate_learning_lstm(16,2,.65,.35,df_KME,1000,'KME Training Set Prediction from Longest Short Term Memory','KME Test Set Prediction from Longest Short Term Memory')

/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


MemoryError: 